In [1]:
import pandas as pd

In [2]:
csv_file_path = "match_results.csv"
df = pd.read_csv(csv_file_path)

In [5]:
artifact_count = df.groupby("vulnerable artifact")["Matching Artifact"].count()

artifact_counts_sorted = artifact_count.sort_values(ascending=False)

In [6]:
print(artifact_counts_sorted)

vulnerable artifact
org.neo4j:neo4j:2.1.0-M01                                558
org.neo4j:neo4j:2.1.0-M02                                558
org.neo4j:neo4j:2.0.3                                    558
org.neo4j:neo4j:2.0.2                                    558
org.neo4j:neo4j:2.0.1                                    558
                                                        ... 
org.eclipse.jetty.http2:http2-server:9.4.2.v20170220       1
org.eclipse.jetty.http2:http2-server:9.4.20.v20190813      1
org.eclipse.jetty.http2:http2-server:9.4.3.v20170317       1
org.eclipse.jetty.http2:http2-server:9.4.7.v20180619       1
org.eclipse.jetty:jetty-http:7.4.3.v20110701               1
Name: Matching Artifact, Length: 1811, dtype: int64


In [15]:
from pymongo import MongoClient


def connect_to_mongodb():
    client = MongoClient("mongodb://localhost:27072/")
    db = client.osv_db
    return db


mongo_db = connect_to_mongodb()


def is_vulnerable(library, version):
    query = {
        "affected.package.name": library,
        "affected.package.ecosystem": "Maven",
        "affected.versions": version,
    }
    return mongo_db.data.find(query)

In [16]:
not_reported_vulnerabilities_count = 0
for index, row in df.iterrows():
    group_id, artifact_id, version = row["Matching Artifact"].split(":")
    library = f"{group_id}:{artifact_id}"
    # print(f"Checking {library} {version}")
    vulnerabilities = list(is_vulnerable(library, version))
    if len(vulnerabilities) == 0:
        not_reported_vulnerabilities_count += 1
        print(f"Vulnerability not reported for {library} {version}")

print(f"Total not reported vulnerabilities: {not_reported_vulnerabilities_count}")

Vulnerability not reported for no.bekk.bekkopen.cde:database 0.2
Vulnerability not reported for no.bekk.bekkopen.cde:database 0.1
Vulnerability not reported for io.github.interestinglab.waterdrop:plugin-spark-source-jdbc 2.0.2
Vulnerability not reported for info.xiaohei.www:spark-database-connector_2.11 1.0.0
Vulnerability not reported for info.xiaohei.www:spark-database-connector_2.10 1.0.0
Vulnerability not reported for org.eclipselabs:net4j.mysql.p2 4.7.2
Vulnerability not reported for org.eclipselabs:net4j.mysql.p2 4.7.0
Vulnerability not reported for org.eclipselabs:net4j.mysql.p2 4.6.2
Vulnerability not reported for org.eclipselabs:net4j.mysql.p2 4.6.0.v20170210-0100
Vulnerability not reported for com.alibaba.ververica:ververica-connector-ads 1.13-vvr-4.0.7
Vulnerability not reported for me.confuser.banmanager.BanManagerLibs:BanManagerLibs 7.9.0
Vulnerability not reported for me.confuser.banmanager.BanManagerLibs:BanManagerLibs 7.7.0
Vulnerability not reported for me.confuser.ban